# LOCO CV to Optimize Ridge Regression Model
Determine whether LOCO CV can improve performance

In [1]:
from sklearn.model_selection import KFold, BaseCrossValidator, cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from scipy.stats import pearsonr
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import os

Set the seed for reproducibility

In [2]:
np.random.seed(1)

## Load in the Data
Readin data from the Supplementary Materials of [Agrawal *et al* 2014](https://link.springer.com/article/10.1186%2F2193-9772-3-8#MOESM1)

In [3]:
data = pd.read_excel(os.path.join('datasets', 'Steel-Dataset.xlsx'))

Get the list of input and output columns

In [4]:
X_cols = data.columns[1:-1]

In [5]:
y_col = data.columns[-1]

## Create the LOCO CV Splitter
Create a class that uses scikit-learn's BaseCrossValidator API, so that it can be used to easily perform cross-validation etc.

In [6]:
class LocoCV(BaseCrossValidator):
    """Generates train/test splits for Leave-One-Cluster-Out cross-validation
    
    Follows the API for scikit-learns cross-validation classes
    
    Parameters
    ----------
    clusterer : ClustererMixin, tool used to generate clusters"""
    
    def __init__(self, clusterer=KMeans()):
        self.clusterer = clusterer
        
    def _iter_test_masks(self, X, y=None, groups=None):
                # Train the clusterer and generate cluster labels
        labels = self.clusterer.fit_predict(X)
        
        # Determine the number of clusters
        clust_labels = np.unique(labels)
        if len(clust_labels) < 2:
            raise ValueError('Clusterer produced < 2 labels. Cannot use for LOCO CV')
            
        # Loop thorugh the clusters
        for label in clust_labels:
            yield labels == label
    
    def get_n_splits(self, X, y=None, groups=None):
        return len(np.unique(self.clusterer.fit_predict(X)))

## Define the KMeans Clusterer
This clusterer will be used for both testing and training

In [7]:
kmeans = Pipeline([
    ('normalize', StandardScaler()),
    ('kmeans', KMeans(n_clusters=10))
])

## Create Ridge Models
Make one model that uses 10-fold CV to set the regularization strength, and another that uses LOCO CV

In [8]:
kfold_ridge = RidgeCV(alphas=np.logspace(-4,4,16), cv=KFold(n_splits=10, shuffle=True))

In [9]:
loco_ridge = RidgeCV(**kfold_ridge.get_params())
loco_ridge.cv = LocoCV(kmeans)

## Test them on the Steel Dataset 
Test each model using both LOCO CV and conventional 10-fold CV to set the ridge regression parameter. We are going to test the performance of the model over 10 iterations of a 10-fold CV and LOCO CV test with k=10.

In [10]:
kfold_R = []
loco_R = []
for i in tqdm(range(10)):
    for train_inds, test_inds in KFold(n_splits=10, shuffle=True).split(data[X_cols], data[y_col]):
        # Make the training and test sets
        train_X = data[X_cols].loc[train_inds]
        test_X = data[X_cols].loc[test_inds]
        train_y = data[y_col].loc[train_inds]
        test_y = data[y_col].loc[test_inds]

        # Test the models
        kfold_ridge.fit(train_X, train_y)
        pred_y = kfold_ridge.predict(test_X)
        kfold_R.append(pearsonr(test_y, pred_y)[0])

        loco_ridge.fit(train_X, train_y)
        pred_y = loco_ridge.predict(test_X)
        loco_R.append(pearsonr(test_y, pred_y)[0])
print('K-fold CV:', np.median(kfold_R), np.std(kfold_R))
print('LOCO CV:', np.median(loco_R), np.std(loco_R))

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))


K-fold CV: 0.9846179274624594 0.006906503097674586
LOCO CV: 0.9832087521161086 0.006898025003811172


*Finding*: The LOCO CV and conventional CV models achieve identical performance on this test

Repeat test using LOCO CV to test model performance

In [11]:
kfold_R = []
loco_R = []
loco_alpha = []
kfold_alpha = []
for i in tqdm(range(10)):
    for train_inds, test_inds in LocoCV(kmeans).split(data[X_cols], data[y_col]):
        # Make the training and test sets
        train_X = data[X_cols].loc[train_inds]
        test_X = data[X_cols].loc[test_inds]
        train_y = data[y_col].loc[train_inds]
        test_y = data[y_col].loc[test_inds]

        # Test the models
        kfold_ridge.fit(train_X, train_y)
        pred_y = kfold_ridge.predict(test_X)
        kfold_R.append(pearsonr(test_y, pred_y)[0])
        kfold_alpha.append(kfold_ridge.alpha_)

        loco_ridge.fit(train_X, train_y)
        pred_y = loco_ridge.predict(test_X)
        loco_R.append(pearsonr(test_y, pred_y)[0])
        loco_alpha.append(loco_ridge.alpha_)
print('K-fold CV:', np.median(kfold_R), np.std(kfold_R))
print('LOCO CV:', np.median(loco_R), np.std(loco_R))

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))


K-fold CV: 0.7980600258847953 0.18128298395240847
LOCO CV: 0.8482634288955642 0.23251481738167534


*Finding*: The performance of both models on the LOCO CV test are worse, but the R score of the model which uses LOCO CV to determine the regularization strength performs better.

In [12]:
print('K-fold alpha:', np.median(kfold_alpha))
print('LOCO alpha:', np.median(loco_alpha))

K-fold alpha: 0.003981071705534973
LOCO alpha: 0.5411695265464638


*Finding*: The LOCO CV model tends to select a stronger regularization parameter